In [1]:
  // обрезать после домена ?


import org.apache.spark.sql._
import org.apache.spark.ml._
import java.io._
import play.api.libs.json._
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._

In [2]:
import java.net.URLDecoder.decode

In [3]:
val vehicleUsers = spark.read.json("/labs/laba02/autousers.json").toDF()

Waiting for a Spark session to start...

vehicleUsers = [autousers: array<string>]


[autousers: array<string>]

In [4]:
val logs = spark.read.option("delimiter", "\t").textFile("/labs/laba02/logs")

logs = [value: string]


[value: string]

In [5]:
logs.show(3, truncate=false)

+-----------------------------------------------------------------------------------------------------+
|value                                                                                                |
+-----------------------------------------------------------------------------------------------------+
|258131083251	1423724291.637	http%3A%2F%2Fwww.zakon.kz%2F                                             |
|258131083251	1423724290.637                                                                          |
|182984926014	1423724291.666	http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5|
+-----------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [6]:
// split df to array rows

val col_ = split(col("value"), "\t").alias("value")
val logs_splited = logs.select(col_)
logs_splited.show(3, truncate=false)

+---------------------------------------------------------------------------------------------------------+
|value                                                                                                    |
+---------------------------------------------------------------------------------------------------------+
|[258131083251, 1423724291.637, http%3A%2F%2Fwww.zakon.kz%2F]                                             |
|[258131083251, 1423724290.637]                                                                           |
|[182984926014, 1423724291.666, http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5]|
+---------------------------------------------------------------------------------------------------------+
only showing top 3 rows



col_ = split(value, 	) AS `value`
logs_splited = [value: array<string>]


[value: array<string>]

In [7]:
// selecting columns by index of arrays

val logs_df = logs_splited.withColumn("UID", $"value"(0))
                    .withColumn("Timestamp", $"value"(1))
                    .withColumn("URL", $"value"(2))
                    .withColumn("Timestamp", to_timestamp(from_unixtime($"Timestamp")))
                    .select("UID","Timestamp", "URL")
                    .filter($"URL".like("%http%"))
//                    .filter($"URL".like("%http_://%"))


logs_df = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [12]:
logs_df.select($"URL").count()

6571047

In [25]:
val logs_df_replaced = logs_df.withColumn("URL", regexp_replace($"URL", lit("u0"), lit("")))withColumn("URL", regexp_replace($"URL", lit("u2"), lit("")))

logs_df_replaced = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [26]:
logs_df_replaced.select($"URL").filter($"URL".like("%u[02]%")).count()

0

In [18]:
// encoding example
"""val url = "http%3A%2F%2Finixi.ru%2Fc246-7497.html"
val s = java.net.URLDecoder.decode(url, "UTF-8")"""

val url = "http%3A%2F%2Finixi.ru%2Fc246-7497.html"
val s = java.net.URLDecoder.decode(url, "UTF-8")


In [27]:
import org.apache.spark.sql.functions.{col, udf}
val urlDecoder = udf((s: String) => java.net.URLDecoder.decode(s, "UTF-8"))


val logs_df_udf = logs_df_replaced.filter($"URL".like("%http%")).withColumn("URL", urlDecoder(col("URL")) as "URL")
logs_df_udf.show(truncate=false)

+------------+-------------------+-------------------------------------------------------+
|UID         |Timestamp          |URL                                                    |
+------------+-------------------+-------------------------------------------------------+
|258131083251|2015-02-12 09:58:11|http://www.zakon.kz/                                   |
|182984926014|2015-02-12 09:58:11|http://www.bmwclub.ru/vb/showthread.php?t=875590&page=5|
|182984926014|2015-02-12 09:58:10|http://www.bmwclub.ru/vb/showthread.php?t=875590&page=4|
|289380960531|2015-02-12 09:58:11|http://www.bmwclub.ru/vb/forumdisplay.php?f=128        |
|289380960531|2015-02-12 09:58:10|http://www.bmwclub.ru/vb/                              |
|85356233460 |2015-02-12 09:58:10|http://www.wmmail.ru/index.php?cf=usf-userfing         |
|204109491902|2015-02-12 09:58:11|https://vk.com/id21743084                              |
|204109491902|2015-02-12 09:58:10|https://vk.com/feed?section=updates                    |

urlDecoder = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
logs_df_udf = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [35]:
val logs_df_domen = logs_df_udf.filter(($"URL".like("%http://%")) || ($"URL".like("%https://%")))
logs_df_domen.show(5, false)


+------------+-------------------+-------------------------------------------------------+
|UID         |Timestamp          |URL                                                    |
+------------+-------------------+-------------------------------------------------------+
|258131083251|2015-02-12 09:58:11|http://www.zakon.kz/                                   |
|182984926014|2015-02-12 09:58:11|http://www.bmwclub.ru/vb/showthread.php?t=875590&page=5|
|182984926014|2015-02-12 09:58:10|http://www.bmwclub.ru/vb/showthread.php?t=875590&page=4|
|289380960531|2015-02-12 09:58:11|http://www.bmwclub.ru/vb/forumdisplay.php?f=128        |
|289380960531|2015-02-12 09:58:10|http://www.bmwclub.ru/vb/                              |
+------------+-------------------+-------------------------------------------------------+
only showing top 5 rows



logs_df_domen = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [42]:
// extracting neat URL

val logs_decoded = logs_df_domen.withColumn("URL", lower(callUDF("parse_url", $"URL", lit("HOST"))))
                               .withColumn("URL", regexp_replace(lower($"URL"), "www.", ""))
logs_decoded.show(5)

+------------+-------------------+----------+
|         UID|          Timestamp|       URL|
+------------+-------------------+----------+
|258131083251|2015-02-12 09:58:11|  zakon.kz|
|182984926014|2015-02-12 09:58:11|bmwclub.ru|
|182984926014|2015-02-12 09:58:10|bmwclub.ru|
|289380960531|2015-02-12 09:58:11|bmwclub.ru|
|289380960531|2015-02-12 09:58:10|bmwclub.ru|
+------------+-------------------+----------+
only showing top 5 rows



logs_decoded = [UID: string, Timestamp: timestamp ... 1 more field]


lastException: Throwable = null
<console>:118: warning: a pure expression does nothing in statement position; you may be omitting necessary parentheses
"""
^


[UID: string, Timestamp: timestamp ... 1 more field]

In [11]:
// extracting neat URL
"""
val substrDel = substring_index($"URL", ".", 1)
val logs_df_clean = logs_df.withColumn("URL", substring_index(substring(regexp_replace($"URL",substrDel , lit("")), 2, 999), "%", 1))

logs_df_clean.show(numRows=5, truncate=200 /*, vertical=true*/)"""

+---+---------+---+
|UID|Timestamp|URL|
+---+---------+---+
+---+---------+---+



substrDel = substring_index(URL, ., 1)
logs_df_clean = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [43]:
logs_decoded.printSchema()

root
 |-- UID: string (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- URL: string (nullable = true)



In [44]:
val vehicleUsers_df = vehicleUsers.select(explode($"autousers").alias("autousers"))
vehicleUsers_df.show(3)

+------------+
|   autousers|
+------------+
|100341861572|
|100473724387|
|100528753939|
+------------+
only showing top 3 rows



vehicleUsers_df = [autousers: string]


[autousers: string]

In [45]:
logs_decoded.select($"UID").distinct.count()

100000

In [46]:
vehicleUsers_df.select("autousers").distinct().count()

5209

In [ ]:
val joined_df = logs_decoded.join(vehicleUsers_df, $"UID" === $"autousers")
joined_df.show()
joined_df.count()

+------------+-------------------+--------------+------------+
|         UID|          Timestamp|           URL|   autousers|
+------------+-------------------+--------------+------------+
|182984926014|2015-02-12 09:58:11|    bmwclub.ru|182984926014|
|182984926014|2015-02-12 09:58:10|    bmwclub.ru|182984926014|
|289380960531|2015-02-12 09:58:11|    bmwclub.ru|289380960531|
|289380960531|2015-02-12 09:58:10|    bmwclub.ru|289380960531|
|289380960531|2015-02-12 09:58:15|    bmwclub.ru|289380960531|
|289380960531|2015-02-12 09:58:14|    bmwclub.ru|289380960531|
|216661893822|2015-02-12 09:58:19|avto-russia.ru|216661893822|
|216661893822|2015-02-12 09:58:18|avto-russia.ru|216661893822|
|220450986911|2015-02-12 09:58:19|    bmwclub.ru|220450986911|
|220450986911|2015-02-12 09:58:18|    bmwclub.ru|220450986911|
| 44329087557|2015-02-12 09:58:20|      inixi.ru| 44329087557|
| 44329087557|2015-02-12 09:58:19|      inixi.ru| 44329087557|
| 10823573029|2015-02-12 09:58:21|       cars.ru| 10823